In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import dotenv
dotenv.load_dotenv('./src_new/azure.env')

from langchain.chat_models import AzureChatOpenAI
# gpt-35-turbo
chat = AzureChatOpenAI(temperature=0, azure_deployment='gpt-4')
chat.predict('''hello''')

'Hello! How can I assist you today?'

In [3]:
from langchain.agents import AgentType, initialize_agent
from langchain.chains import LLMMathChain
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from src_new.tools import TreeAPI

In [4]:
from langchain.prompts import PromptTemplate
react_tmpl_zh = PromptTemplate.from_template('''尽可能回答以下问题。 你可以使用以下工具：

{tools}

使用这些工具的方式是指定 json blob。
具体来说，这个 json 应该有一个"action"键（包含要使用的工具的名称）和一个"action_input"键（此处包含工具的输入）。

"action"字段的值**只能**从下列名称中选取：{tool_names}

$JSON_BLOB 应该只包含**单个**操作，**不要**返回多个操作的列表。 以下是有效 $JSON_BLOB 的示例：

```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```

你应当**始终**使用如下格式：

Question: 你必须回答的输入问题
Thought: 你应该时刻思考该做什么
Action:
```
$JSON_BLOB
```
Observation: 行动的结果
...（这个 Thought/Action/Observation 可以重复N次)
Thought: 我现在知道了最终答案
Final Answer: 原始输入问题的最终答案

开始！

Question: {input}
Thought: {agent_scratchpad}''')

In [5]:
from langchain import hub
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.tools.render import render_text_description
from langchain.agents.output_parsers import ReActJsonSingleInputOutputParser

In [79]:
from src_new.tools import return_tools, reset_tool
tools = return_tools()
tools

[StructuredTool(name='get_children', description='get_children() - Retrieves children nodes of the current position of the tree.\n    Args:\n        None\n    Returns:\n        result: The content of  the children nodes', args_schema=<class 'pydantic.main.get_childrenSchemaSchema'>, func=<function get_children at 0x7f0be6daadc0>),
 StructuredTool(name='search_given_children', description="search_given_children(k: Union[List[int], str], text) - If you are not sure which child will contain the given text, you can use this tool to search which child will have the related information.    If the given node have children nodes, the tool will search the children of the given child-node to determine the relation. Or it only considers the given nodes.    If there are many children nodes, use `all` mode to search **before** you decide on yourself. You should vist **at least** the top-2 choices to draw a conclusion\n    Args: \n        k: The indices of children you want to query, it can be List[

In [50]:

prompt = react_tmpl_zh.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)

prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'tools': "get_children: get_children() - Retrieves children nodes of the current position of the tree.\n    Args:\n        None\n    Returns:\n        result: The content of  the children nodes\nsearch_given_children: search_given_children(k: Union[List[int], str], text) - If you are not sure which child will contain the given text, you can use this tool to search which child will have the related information.    If the given node have children nodes, the tool will search the children of the given child-node to determine the relation. Or it only considers the given nodes.    If there are many children nodes, use `all` mode to search **before** you decide on yourself. You should vist **at least** the top-2 choices to draw a conclusion\n    Args: \n        k: The indices of children you want to query, it can be List[int] or str('all').\n    Returns:\n        result:dict : The top four relevant nodes you get.

In [34]:
chat_model_with_stop = chat.bind(stop=["\nObservation"])

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | chat_model_with_stop
    | ReActJsonSingleInputOutputParser()
)

In [80]:
from langchain.agents import AgentExecutor
from langchain.agents import AgentType, Tool, initialize_agent
agent_executor = AgentExecutor(tools=tools,agent=agent,verbose=True, handle_parsing_errors=True,max_iterations=50)

In [83]:
system_prompt = '你现在可以访问一棵文档树且站在根节点上,你可以通过工具访问这个文档树并抽取相关的信息回答下面这个问题'
question = 'Material Class shall be determined by the user/purchaser. What information should they apply? 我建议你查看AnnexB之后再回答'

In [84]:
qurey = system_prompt + question
reset_tool()
agent_executor.invoke({
    "input": qurey
})



> Entering new AgentExecutor chain...
Could not parse LLM output: 我需要找到 AnnexB 的内容，然后从中找到用户/购买者应该应用的信息来确定 Material Class。首先，我需要查看当前节点的子节点，看看是否可以找到 AnnexB。Invalid or incomplete response我需要使用 get_children 工具来查看当前节点的子节点。 
Action:
```
{
  "action": "get_children",
  "action_input": {}
}
```The results are formated by {index}:{content}. 0: Specification for Subsea Wellhead and Tree Equipment  1: API SPECIFICATION 17D THIRD EDITION, OCTOBER 2021  2: API MONOGRAM PROGRAM EFFECTIVE DATE: OCTOBER 2022  3:  4: Special Notes  5: Foreword  6: Contents  7: Figures  8: Tables  9: Specification for Subsea Wellhead and Tree Equipment  10: 1 Scope  11: 2 Normative References  12: 3 Terms, Definitions, Acronyms, Abbreviations, and Symbols  13: 4 Application, Service Conditions, and Production Specification Levels  14: 5 Common System Requirements  15: 6 General Design Requirements for Subsea Tree Systems  16: 7 Specific Requirements—Subsea Tree-related Equipment and Subassemblies  17: 8 Specific Requi

{'input': '你现在可以访问一棵文档树且站在根节点上,你可以通过工具访问这个文档树并抽取相关的信息回答下面这个问题Material Class shall be determined by the user/purchaser. What information should they apply? 我建议你查看AnnexB之后再回答',
 'output': '用户/购买者应该应用 API 6A 和 Table 1 中的制造要求来确定 Material Class。'}

In [ ]:
# TODO: parse table 
# TODO: add globle search